In [ ]:
push!(LOAD_PATH, "/home/nicolelislab/SpikeSorting.jl/")

In [ ]:
#addprocs()
using rhd2000evalboard, HDF5, rhd2000registers, SortSpikes, DistributedArrays, ExtractSpikes

In [ ]:
#Opal Kelly XEM6010 board
open_board()

# Load Rhythm FPGA configuration bitfile (provided by Intan Technologies).
uploadFpgaBitfile();

#Initialize board
initialize_board()

#For 64 channel need two data streams, and data will come in 
#on the rising AND falling edges of SCLK
enableDataStream(1, true)
setDataSource(0, 0) #port A MISO1
setDataSource(1, 8) #port A MISO1 DDR
println(rhd2000evalboard.dataStreamEnabled)
println(rhd2000evalboard.numDataStreams)

#Select per-channel amplifier sampling rate
setSampleRate(20000)

#Now that we have set our sampling rate, we can set the MISO sampling delay
#which is dependent on the sample rate. We use a 6.0 foot cable
setCableLengthFeet("PortA", 6.0)

# Let's turn one LED on to indicate that the program is running.
ledArray=[1,1,0,0,0,0,0,0]
setLedDisplay(ledArray)

In [ ]:
#Set up an RHD2000 register object using this sample rate 
#to optimize MUX-related register settings.
r=CreateRHD2000Registers(20000.0)

#First, let's create a command list for the AuxCmd1 slot.  This command
#sequence will create a 1 kHz, full-scale sine wave for impedance testing.

#Next, we'll create a command list for the AuxCmd2 slot.  This command sequence
#will sample the temperature sensor and other auxiliary ADC inputs.

#For the AuxCmd3 slot, we will create two command sequences.  Both sequences
#will configure and read back the RHD2000 chip registers, but one sequence will
#also run ADC calibration.

#Before generating register configuration command sequences, set amplifier
#bandwidth paramters.


#Upload version with no ADC calibration to AuxCmd3 RAM Bank 0.
commandList=createCommandListRegisterConfig(zeros(Int32,1),false,r)
uploadCommandList(commandList, "AuxCmd3", 0)

#Upload version with ADC calibration to AuxCmd3 RAM Bank 1.
commandList=createCommandListRegisterConfig(zeros(Int32,1),true,r)
uploadCommandList(commandList, "AuxCmd3", 1)

selectAuxCommandLength("AuxCmd3", 0, length(commandList) - 1)

#Select RAM Bank 1 for AuxCmd3 initially, so the ADC is calibrated.
selectAuxCommandBank("PortA", "AuxCmd3", 1);

setMaxTimeStep(60)
setContinuousRunMode(false)
runBoard()

while (isRunning())
end

flushBoard()

selectAuxCommandBank("PortA", "AuxCmd3", 0)

In [ ]:
#1 second sequence for calibration
setMaxTimeStep(600) #set num samples to acquire
setContinuousRunMode(false)
mytime=zeros(Int32,2)
st=Array(Sorting,64)
for i=1:64
    st[i]=Sorting(SpikeDetection(),Cluster(),zeros(Int64,600),zeros(Int,500),zeros(Int,500),2,zeros(Int,50,100))
end
s=distribute(st);

In [ ]:
usbDataRead=true

runBoard()

while (usbDataRead || isRunning())
    #If Fifo contains words, they are read
    usbDataRead = readDataBlocks(1,mytime,s,"METHOD_SORTCAL")  
end

flushBoard()

In [ ]:
setMaxTimeStep(600) #set num samples to acquire

#prepare output file
saveOut="test.jld"
h5open(saveOut, "w") do myfile
    d=d_create(myfile, "time", Int32, ((1,),(-1,)), "chunk", (100,))
    d[1]=1
    for i=1:64
        e=d_create(myfile, string(i), Int64, ((1,),(-1,)),"chunk", (100,))
        e=d_create(myfile, string("n",i), Int64, ((1,),(-1,)),"chunk", (100,))
    end
end

In [ ]:
runBoard()

while (usbDataRead || isRunning())
    usbDataRead = readDataBlocks(1,mytime,s,"METHOD_SORT")
    if (length(mytime) >= 3000)
        queueToFile(mytime,s,saveOut)
    end
    
end

queueToFile(mytime,s,saveOut)

flushBoard()

In [ ]:
#print the output of a set of time stamps to make sure they are not mistaken

h5open(saveOut) do hi
    e=d_open(hi, "time")
    println(e[:])
end